In [1]:
!pip install datasets pillow pandas tqdm

### 25 Selected Classes with COCO Category IDs

**Vehicles:** car(3), truck(8), bus(6), motorcycle(4), bicycle(2), airplane(5)  
**Person:** person(1)  
**Outdoor:** traffic light(10), stop sign(13), bench(15)  
**Animals:** dog(18), cat(17), horse(19), bird(16), cow(21), elephant(22)  
**Kitchen & Food:** bottle(44), cup(47), bowl(51), pizza(59), cake(61)  
**Furniture:** chair(62), couch(63), bed(65), potted plant(64)

In [2]:
# 25 Selected Classes (CORRECT indices from detection-datasets/coco)

SELECTED_CLASSES = {
    'person': 0,
    'bicycle': 1,
    'car': 2,
    'motorcycle': 3,
    'airplane': 4,
    'bus': 5,
    'train': 6,
    'truck': 7,
    'traffic light': 9,
    'stop sign': 11,
    'bench': 13,
    'bird': 14,
    'cat': 15,
    'dog': 16,
    'horse': 17,
    'cow': 19,
    'elephant': 20,
    'bottle': 39,
    'cup': 41,
    'bowl': 45,
    'pizza': 53,
    'cake': 55,
    'chair': 56,
    'couch': 57,
    'potted plant': 58,
    'bed': 59
}

IMAGES_PER_CLASS = 100
BASE_DIR = "smartvision_dataset"

In [3]:
## STEP 1: IMPORTS AND CONFIGURATION

import os
import json
from datasets import load_dataset
from PIL import Image
from collections import defaultdict
from tqdm import tqdm
import random

In [4]:
## STEP 2: LOAD COCO DATASET FROM HUGGING FACE

print("📥 Loading COCO dataset in STREAMING mode (no download)...")
dataset = load_dataset("detection-datasets/coco", split="train", streaming=True)
print("✅ Dataset loaded in streaming mode!")

📥 Loading COCO dataset in STREAMING mode (no download)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

✅ Dataset loaded in streaming mode!


In [5]:
print("\n🔍 Starting image collection from COCO dataset stream...")
print(f"🎯 Target: {IMAGES_PER_CLASS} images per class\n")

# Storage
class_images = {cls: [] for cls in SELECTED_CLASSES}
class_counts = {cls: 0 for cls in SELECTED_CLASSES}

images_processed = 0
max_iterations = 200000  # safety limit

for sample in tqdm(dataset):

    images_processed += 1
    if images_processed > max_iterations:
        print("⚠️ Safety limit reached!")
        break

    image = sample["image"]
    annotations = sample["objects"]

    for label, bbox in zip(annotations["category"], annotations["bbox"]):

        for class_name, class_id in SELECTED_CLASSES.items():

            if class_counts[class_name] >= IMAGES_PER_CLASS:
                continue

            if label == class_id:
                class_images[class_name].append({
                    "image": image,
                    "bbox": bbox,
                    "label": label
                })
                class_counts[class_name] += 1

    # Stop when all classes have enough images
    if all(count >= IMAGES_PER_CLASS for count in class_counts.values()):
        print("\n✅ Collected 100 images for ALL classes!")
        break

print("\n📊 Collection Summary:")
for cls, count in class_counts.items():
    print(f"{cls:15s} → {count}")



🔍 Starting image collection from COCO dataset stream...
🎯 Target: 100 images per class



5977it [04:51, 20.53it/s]


✅ Collected 100 images for ALL classes!

📊 Collection Summary:
person          → 100
bicycle         → 100
car             → 100
motorcycle      → 100
airplane        → 100
bus             → 100
train           → 100
truck           → 100
traffic light   → 100
stop sign       → 100
bench           → 100
bird            → 100
cat             → 100
dog             → 100
horse           → 100
cow             → 100
elephant        → 100
bottle          → 100
cup             → 100
bowl            → 100
pizza           → 100
cake            → 100
chair           → 100
couch           → 100
potted plant    → 100
bed             → 100


In [6]:
## STEP 4: CREATE FOLDER STRUCTURE

print("\n📁 Creating project folder structure...")
print()

# Create main directory
os.makedirs(BASE_DIR, exist_ok=True)

# Create subdirectories for Classification task
os.makedirs(f"{BASE_DIR}/classification/train", exist_ok=True)
os.makedirs(f"{BASE_DIR}/classification/val", exist_ok=True)
os.makedirs(f"{BASE_DIR}/classification/test", exist_ok=True)

# Create subdirectories for Detection task
os.makedirs(f"{BASE_DIR}/detection/images", exist_ok=True)
os.makedirs(f"{BASE_DIR}/detection/labels", exist_ok=True)

# Create class folders inside train/val/test
for class_name in SELECTED_CLASSES.keys():
    os.makedirs(f"{BASE_DIR}/classification/train/{class_name}", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/classification/val/{class_name}", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/classification/test/{class_name}", exist_ok=True)

print("✅ Folder structure created successfully!")
print()
print("📂 Structure:")
print(f"""
{BASE_DIR}/
├── classification/
│   ├── train/
│   │   ├── person/
│   │   ├── car/
│   │   └── ... (25 class folders)
│   ├── val/
│   │   └── ... (25 class folders)
│   └── test/
│       └── ... (25 class folders)
│
└── detection/
    ├── images/
    └── labels/
""")


📁 Creating project folder structure...

✅ Folder structure created successfully!

📂 Structure:

smartvision_dataset/
├── classification/
│   ├── train/
│   │   ├── person/
│   │   ├── car/
│   │   └── ... (25 class folders)
│   ├── val/
│   │   └── ... (25 class folders)
│   └── test/
│       └── ... (25 class folders)
│
└── detection/
    ├── images/
    └── labels/



In [7]:
## STEP 5: TRAIN/VAL/TEST SPLIT (70/15/15)

print("="*70)
print("🔀 Preparing Train/Val/Test splits...")
print("📊 Split Ratio: 70% Train / 15% Val / 15% Test")
print("="*70)
print()

# Initialize metadata dictionary
metadata = {
    'total_images': 0,
    'classes': {},
    'splits': {'train': 0, 'val': 0, 'test': 0}
}

# Create split dictionaries for each class
train_data = {}
val_data = {}
test_data = {}

# Process each class
for class_name in SELECTED_CLASSES.keys():

    all_items = class_images.get(class_name, [])

    if not all_items:
        print(f"⚠️ Warning: No images found for {class_name}")
        continue

    # Calculate split indices
    n = len(all_items)
    train_split = int(0.7 * n)   # 70% for training
    val_split = int(0.85 * n)    # 15% for validation
    # Remaining 15% for test

    # Split the data
    train_data[class_name] = all_items[:train_split]
    val_data[class_name] = all_items[train_split:val_split]
    test_data[class_name] = all_items[val_split:]

    # Store split info in metadata
    metadata['classes'][class_name] = {
        'train': len(train_data[class_name]),
        'val': len(val_data[class_name]),
        'test': len(test_data[class_name]),
        'total': len(all_items)
    }

    metadata['splits']['train'] += len(train_data[class_name])
    metadata['splits']['val'] += len(val_data[class_name])
    metadata['splits']['test'] += len(test_data[class_name])
    metadata['total_images'] += len(all_items)

    print(f"{class_name:20s}: Train={len(train_data[class_name]):3d} | Val={len(val_data[class_name]):2d} | Test={len(test_data[class_name]):2d}")

🔀 Preparing Train/Val/Test splits...
📊 Split Ratio: 70% Train / 15% Val / 15% Test

person              : Train= 70 | Val=15 | Test=15
bicycle             : Train= 70 | Val=15 | Test=15
car                 : Train= 70 | Val=15 | Test=15
motorcycle          : Train= 70 | Val=15 | Test=15
airplane            : Train= 70 | Val=15 | Test=15
bus                 : Train= 70 | Val=15 | Test=15
train               : Train= 70 | Val=15 | Test=15
truck               : Train= 70 | Val=15 | Test=15
traffic light       : Train= 70 | Val=15 | Test=15
stop sign           : Train= 70 | Val=15 | Test=15
bench               : Train= 70 | Val=15 | Test=15
bird                : Train= 70 | Val=15 | Test=15
cat                 : Train= 70 | Val=15 | Test=15
dog                 : Train= 70 | Val=15 | Test=15
horse               : Train= 70 | Val=15 | Test=15
cow                 : Train= 70 | Val=15 | Test=15
elephant            : Train= 70 | Val=15 | Test=15
bottle              : Train= 70 | Val=15 | Test=1

In [11]:
import os
from PIL import Image
from tqdm import tqdm

print("="*70)
print("💾 STEP 6: SAVING IMAGES TO DISK")
print("="*70)
print()

# PART A: SAVE CLASSIFICATION IMAGES (FROM DETECTION DATA)

print("📁 PART A: Saving Classification Images...")
print("   Format: Cropped objects, 224x224 pixels\n")

classification_stats = {'train': 0, 'val': 0, 'test': 0}

# Process each split
for split_name, split_data in [('train', train_data), ('val', val_data), ('test', test_data)]:

    print(f"📂 Processing {split_name.upper()} split...")

    # Process each class
    for class_name, items in tqdm(split_data.items(), desc=f"  {split_name}"):

        class_folder = f"{BASE_DIR}/classification/{split_name}/{class_name}"
        os.makedirs(class_folder, exist_ok=True)

        # Save each image
        for img_idx, item in enumerate(items):

            img = item['image']
            bbox = item['bbox']          # ✅ FIX
            class_id = item['label']     # ✅ FIX

            try:
                x, y, w, h = bbox

                # Crop and resize
                cropped_img = img.crop((x, y, x + w, y + h))
                cropped_img = cropped_img.resize((224, 224), Image.LANCZOS)

                # Save
                img_filename = f"{class_name}_{split_name}_{img_idx:04d}.jpg"
                img_path = os.path.join(class_folder, img_filename)
                cropped_img.save(img_path, quality=95)

                classification_stats[split_name] += 1

            except Exception as e:
                print(f"⚠️ Error: {class_name} image {img_idx}: {e}")

print()
print("="*70)
print("✅ CLASSIFICATION IMAGES SAVED!")
print("="*70)
print(f"📊 Train: {classification_stats['train']} images")
print(f"📊 Val:   {classification_stats['val']} images")
print(f"📊 Test:  {classification_stats['test']} images")
print(f"📊 Total: {sum(classification_stats.values())} images")
print()


💾 STEP 6: SAVING IMAGES TO DISK

📁 PART A: Saving Classification Images...
   Format: Cropped objects, 224x224 pixels

📂 Processing TRAIN split...


  train: 100%|██████████| 26/26 [00:04<00:00,  5.85it/s]


📂 Processing VAL split...


  val: 100%|██████████| 26/26 [00:00<00:00, 26.61it/s]


📂 Processing TEST split...


  test: 100%|██████████| 26/26 [00:01<00:00, 25.83it/s]


✅ CLASSIFICATION IMAGES SAVED!
📊 Train: 1820 images
📊 Val:   390 images
📊 Test:  390 images
📊 Total: 2600 images



In [13]:
# PART B: SAVE DETECTION IMAGES (YOLO FORMAT)

print("="*70)
print("📁 PART B: Saving Detection Images & Annotations...")
print("   Format: Full images with YOLO .txt labels\n")

detection_stats = {'images': 0, 'annotations': 0, 'objects': 0}

# COCO → YOLO class mapping
coco_to_yolo = {class_id: idx for idx, class_id in enumerate(SELECTED_CLASSES.values())}

# Combine train + val for detection
all_detection_data = []
for class_name in SELECTED_CLASSES.keys():
    all_detection_data.extend(train_data.get(class_name, []))
    all_detection_data.extend(val_data.get(class_name, []))

print(f"📊 Total detection images: {len(all_detection_data)}\n")

# Save images and YOLO labels
for img_idx, item in enumerate(tqdm(all_detection_data, desc="Saving detection data")):

    img = item['image']
    img_width, img_height = img.size

    # Save full image
    img_filename = f"image_{img_idx:06d}.jpg"
    img_path = os.path.join(f"{BASE_DIR}/detection/images", img_filename)
    img.save(img_path, quality=95)
    detection_stats['images'] += 1

    # ✅ FIX: use bbox + label
    bbox = item['bbox']
    cat_id = item['label']

    # Convert to YOLO format
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    w_norm = w / img_width
    h_norm = h / img_height

    yolo_class_id = coco_to_yolo[cat_id]

    yolo_line = f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"

    # Save label file
    label_filename = f"image_{img_idx:06d}.txt"
    label_path = os.path.join(f"{BASE_DIR}/detection/labels", label_filename)

    with open(label_path, 'w') as f:
        f.write(yolo_line)

    detection_stats['annotations'] += 1
    detection_stats['objects'] += 1

print()
print("="*70)
print("✅ DETECTION DATASET CREATED!")
print("="*70)
print(f"📊 Images:     {detection_stats['images']}")
print(f"📊 Labels:     {detection_stats['annotations']}")
print(f"📊 Objects:    {detection_stats['objects']}")
print(f"📊 Avg/image:  {detection_stats['objects']/detection_stats['images']:.2f}")
print()


📁 PART B: Saving Detection Images & Annotations...
   Format: Full images with YOLO .txt labels

📊 Total detection images: 2210



Saving detection data: 100%|██████████| 2210/2210 [00:03<00:00, 676.61it/s]


✅ DETECTION DATASET CREATED!
📊 Images:     2210
📊 Labels:     2210
📊 Objects:    2210
📊 Avg/image:  1.00



In [14]:
# PART C: CREATE YOLO CONFIG FILE

print("📝 Creating YOLO configuration file...\n")

yaml_content = f"""# SmartVision Dataset - YOLOv8 Configuration
path: {os.path.abspath(BASE_DIR)}/detection
train: images
val: images

names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: traffic light
  9: stop sign
  10: bench
  11: bird
  12: cat
  13: dog
  14: horse
  15: cow
  16: elephant
  17: bottle
  18: cup
  19: bowl
  20: pizza
  21: cake
  22: chair
  23: couch
  24: potted plant
  25: bed

nc: 26
"""

yaml_path = f"{BASE_DIR}/detection/data.yaml"
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ Created: {yaml_path}\n")

📝 Creating YOLO configuration file...

✅ Created: smartvision_dataset/detection/data.yaml



In [15]:
# PART D: SAVE METADATA

print("📊 Saving metadata...\n")

metadata['classification'] = classification_stats
metadata['detection'] = detection_stats
metadata['dataset_path'] = os.path.abspath(BASE_DIR)

metadata_path = f"{BASE_DIR}/dataset_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, indent=2, fp=f)

print(f"✅ Saved: {metadata_path}\n")

📊 Saving metadata...

✅ Saved: smartvision_dataset/dataset_metadata.json



In [16]:
print("="*70)
print("🎉 DATASET SETUP COMPLETE!")
print("="*70)
print()
print(f"📁 Location: {os.path.abspath(BASE_DIR)}")
print()
print("📂 Classification Dataset:")
print(f"   ├─ Train:  {classification_stats['train']} images (70%)")
print(f"   ├─ Val:    {classification_stats['val']} images (15%)")
print(f"   ├─ Test:   {classification_stats['test']} images (15%)")
print(f"   └─ Total:  {sum(classification_stats.values())} cropped images (224x224)")
print()
print("📂 Detection Dataset:")
print(f"   ├─ Images: {detection_stats['images']} full images")
print(f"   ├─ Labels: {detection_stats['annotations']} YOLO .txt files")
print(f"   └─ Objects: {detection_stats['objects']} annotated objects")
print()
print("="*70)
print("✅ LEARNERS CAN NOW START:")
print("="*70)
print("Step 7:  Exploratory Data Analysis (EDA)")
print("Step 8:  Train Classification Models")
print("Step 9:  Train YOLO Detection Model")
print("Step 10: Build Streamlit Application")
print("Step 11: Deploy to Hugging Face Spaces")
print("="*70)

🎉 DATASET SETUP COMPLETE!

📁 Location: /content/smartvision_dataset

📂 Classification Dataset:
   ├─ Train:  1820 images (70%)
   ├─ Val:    390 images (15%)
   ├─ Test:   390 images (15%)
   └─ Total:  2600 cropped images (224x224)

📂 Detection Dataset:
   ├─ Images: 2210 full images
   ├─ Labels: 2210 YOLO .txt files
   └─ Objects: 2210 annotated objects

✅ LEARNERS CAN NOW START:
Step 7:  Exploratory Data Analysis (EDA)
Step 8:  Train Classification Models
Step 9:  Train YOLO Detection Model
Step 10: Build Streamlit Application
Step 11: Deploy to Hugging Face Spaces


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
import os

save_path = "/content/drive/MyDrive/datasets/huggingface_images"
os.makedirs(save_path, exist_ok=True)


In [ ]:
!ls /content


In [ ]:
!mv /content/smartvision_dataset/ /content/drive/MyDrive/